In [34]:
import pandas as pd
import nbformat
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge

import seaborn as sns

In [35]:
import nbformat

# Load the provided Jupyter notebook
notebook_content = None
with open("MLR_ElasticNet_class4.ipynb", "r", encoding="utf-8") as file:
    notebook_content = nbformat.read(file, as_version=4)

# Extract all the code cells from the notebook
code_cells = [cell['source'] for cell in notebook_content.cells if cell.cell_type == 'code']

# Display the first few code cells for review
code_cells[:3]  # Displaying first 3 cells for brevity. We can check more if needed.


['import numpy as np\nimport glob\nimport datetime\nfrom datetime import date\nfrom datetime import timedelta\nimport datetime\nfrom dateutil.relativedelta import relativedelta\nfrom datetime import datetime\nimport pandas as pd\nimport os\nfrom netCDF4 import Dataset\nimport netCDF4 as nc\nimport matplotlib.pyplot as plt# importing modules and packages\nimport pandas as pd\nimport numpy as np\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.linear_model import LinearRegression\nfrom sklearn.metrics import mean_squared_error, mean_absolute_error\nfrom sklearn import preprocessing',
 "list_of_files = glob.glob('Daily_all_feature_BUFN_GL_nomissing.txt')  # create the list of file\n",
 'list_of_files']

In [36]:
# Load the provided dataset
data_path = "Daily_all_feature_BARENT_nomissing.txt"
df = pd.read_csv(data_path, delim_whitespace=True, header=None)

# Rename columns based on the code from the notebook
df.columns = ['year', 'mon', 'day', 'swd', 'swu', 'lwd', 'lwu', 'ppt', 't', 'q', 'u', 'v', 'aod', 'bc', 'ice']

# Display the first few rows of the dataset
df.head()


,year,mon,day,swd,swu,lwd,lwu,ppt,t,q,u,v,aod,bc,ice
0,2001.0,1.0,1.0,0.0,0.0,202.725,299.017,2.09119,268.73602,0.00229,7.88108,-0.19518,0.15840,-9999.0,551777.11
1,2001.0,1.0,2.0,0.0,0.0,210.973,303.979,5.52125,270.31927,0.00271,8.04636,-0.06832,0.16207,-9999.0,552360.97
2,2001.0,1.0,3.0,0.0,0.0,212.577,303.251,2.20616,270.71173,0.00269,4.95324,0.25475,0.12164,-9999.0,543476.12
3,2001.0,1.0,4.0,0.0,0.0,219.647,305.138,8.16399,271.53030,0.00292,7.51686,0.42577,0.14539,-9999.0,522255.70
4,2001.0,1.0,5.0,0.0,0.0,217.732,305.322,2.37642,271.90396,0.00302,5.70449,0.44863,0.15322,-9999.0,494477.31


Q) Develop MLR and Elastic Net Models


In [37]:


# Splitting the data into training and testing sets (80% train, 20% test)
X = df.iloc[:, :-1]  # Features (excluding the last column)
y = df.iloc[:, -1]   # Target variable (the last column)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the MLR model
mlr_model = LinearRegression()
mlr_model.fit(X_train, y_train)

# Predict using the MLR model
mlr_train_pred = mlr_model.predict(X_train)
mlr_test_pred = mlr_model.predict(X_test)

# Calculate training and testing RMSE for MLR model
mlr_train_rmse = np.sqrt(mean_squared_error(y_train, mlr_train_pred))
mlr_test_rmse = np.sqrt(mean_squared_error(y_test, mlr_test_pred))

mlr_train_rmse, mlr_test_rmse


(73994.4812616219, 72673.55621699484)

Q) Interpretation of Coefficients of lwu and lwd

In [38]:
# Splitting the data into features (X) and target variable (y)
X_df = df.drop(columns=['ice'])
y_df = df['ice']

# Splitting the data into training and testing sets (80% training and 20% testing)
X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(X_df, y_df, test_size=0.2, random_state=42)

# Building and training the MLR model using the loaded dataset
mlr_model_df = LinearRegression()
mlr_model_df.fit(X_train_df, y_train_df)

# Extracting coefficients for 'lwu' and 'lwd' from the MLR model
lwu_coefficient_df = mlr_model_df.coef_[X_df.columns.get_loc("lwu")]
lwd_coefficient_df = mlr_model_df.coef_[X_df.columns.get_loc("lwd")]

lwu_coefficient_df, lwd_coefficient_df


(-18701.64974021368, 4085.197064814196)

Interpretation:

- The negative coefficient for lwu suggests that as the value of lwu increases, the dependent variable ice tends to decrease. The magnitude of the coefficient indicates the strength of this inverse relationship.
- The positive coefficient for lwd suggests that as the value of lwd increases, the dependent variable ice also tends to increase.

In [39]:
# Identifying the most important features based on the MLR coefficients for the loaded dataset with named columns
importance_mlr_df = pd.DataFrame({
    'Feature': X_df.columns,
    'MLR_Coefficient': mlr_model_df.coef_,
    'Absolute_MLR_Coefficient': np.abs(mlr_model_df.coef_)
})

# Sorting by absolute coefficient value to determine importance
important_features_mlr_df = importance_mlr_df.sort_values(by='Absolute_MLR_Coefficient', ascending=False)
important_features_mlr_df


,Feature,MLR_Coefficient,Absolute_MLR_Coefficient
9,q,2.487675e+07,2.487675e+07
12,aod,2.310897e+05,2.310897e+05
6,lwu,-1.870165e+04,1.870165e+04
1,mon,-9.148644e+03,9.148644e+03
4,swu,4.947334e+03,4.947334e+03
0,year,-4.770781e+03,4.770781e+03
5,lwd,4.085197e+03,4.085197e+03
7,ppt,-3.649933e+03,3.649933e+03
11,v,-2.519824e+03,2.519824e+03
10,u,1.844047e+03,1.844047e+03


Observations:

- The feature q has the highest absolute coefficient, making it the most influential feature in predicting the dependent variable ice in the MLR model.
- Features like aod, lwu, and mon also have high absolute coefficients, indicating their significant influence on the target variable ice.

Q) Coefficient Shrinkage in Ridge Regression

In [40]:
# Building and training the Ridge regression model
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train, y_train)

# Extracting the coefficients from the Ridge model
ridge_coefficients = ridge_model.coef_

# Creating a DataFrame to compare the coefficients of MLR and Ridge models
coeff_comparison_ridge = pd.DataFrame({
    'Feature': X_df.columns,
    'MLR_Coefficient': mlr_model.coef_,
    'Ridge_Coefficient': ridge_coefficients
})

coeff_comparison_ridge['Difference'] = coeff_comparison_ridge['MLR_Coefficient'] - coeff_comparison_ridge['Ridge_Coefficient']

coeff_comparison_ridge


,Feature,MLR_Coefficient,Ridge_Coefficient,Difference
0,year,-4.770781e+03,-4749.420352,-2.136091e+01
1,mon,-9.148644e+03,-9456.377824,3.077339e+02
2,day,-1.172969e+02,-110.623593,-6.673314e+00
3,swd,-4.073610e+02,-423.579606,1.621857e+01
4,swu,4.947334e+03,4864.179971,8.315430e+01
5,lwd,4.085197e+03,5249.688391,-1.164491e+03
6,lwu,-1.870165e+04,-18882.718853,1.810691e+02
7,ppt,-3.649933e+03,-3967.936594,3.180034e+02
8,t,1.545832e+03,2169.278755,-6.234469e+02
9,q,2.487675e+07,5795.099972,2.487096e+07


Observations:

- There's noticeable shrinkage in coefficients when using Ridge regression compared to the MLR model. For instance, the coefficient of the feature q in the MLR model is substantially larger than its corresponding coefficient in the Ridge model.
- This shrinkage is due to the L2 regularization applied in Ridge regression, which penalizes large coefficients to prevent overfitting and make the model more generalizable.

Q) Based on teh test and training accuracy, would you use the ridge regression model to predict your dependent variable "ice"?

In [41]:
# Predictions using Ridge model on training and test data
ridge_train_preds = ridge_model.predict(X_train)
ridge_test_preds = ridge_model.predict(X_test)

# Calculating MAE for the Ridge model
ridge_train_mae = mean_absolute_error(y_train, ridge_train_preds)
ridge_test_mae = mean_absolute_error(y_test, ridge_test_preds)

ridge_performance = pd.DataFrame({
    'Dataset': ['Training', 'Test'],
    'MLR MAE': [mean_absolute_error(y_train, mlr_model.predict(X_train)), mean_absolute_error(y_test, mlr_model.predict(X_test))],
    'Ridge MAE': [ridge_train_mae, ridge_test_mae]
})

ridge_performance


,Dataset,MLR MAE,Ridge MAE
0,Training,57975.381730,58188.795000
1,Test,56475.947226,56329.397133


Observations:

- The Ridge model has slightly higher training error compared to the MLR model, suggesting a slight underfitting on the training data.
- However, the Ridge model performs better on the test data compared to the MLR model, indicating better generalization to unseen data.

Q) Repeat your analysis based on training teh first 80% of teh data nd testing with last 20% of the data.

In [42]:
# Splitting the data based on the first 80% for training and the last 20% for testing
train_size_df = int(0.8 * len(df))
X_train_80, X_test_20 = X_df.iloc[:train_size_df], X_df.iloc[train_size_df:]
y_train_80, y_test_20 = y_df.iloc[:train_size_df], y_df.iloc[train_size_df:]

# Retraining the MLR and Ridge models on the new training set
mlr_model_80 = LinearRegression()
mlr_model_80.fit(X_train_80, y_train_80)

ridge_model_80 = Ridge(alpha=1.0)
ridge_model_80.fit(X_train_80, y_train_80)

# Calculating MAE for both models on the new training and test data
mlr_train_mae_80 = mean_absolute_error(y_train_80, mlr_model_80.predict(X_train_80))
mlr_test_mae_20 = mean_absolute_error(y_test_20, mlr_model_80.predict(X_test_20))

ridge_train_mae_80 = mean_absolute_error(y_train_80, ridge_model_80.predict(X_train_80))
ridge_test_mae_20 = mean_absolute_error(y_test_20, ridge_model_80.predict(X_test_20))

accuracy_comparison_80_20 = pd.DataFrame({
    'Model': ['MLR', 'Ridge'],
    'Train MAE (80%)': [mlr_train_mae_80, ridge_train_mae_80],
    'Test MAE (20%)': [mlr_test_mae_20, ridge_test_mae_20]
})

accuracy_comparison_80_20


,Model,Train MAE (80%),Test MAE (20%)
0,MLR,57039.574467,69493.484717
1,Ridge,57107.345371,69731.081921


Analysis:

- Both models perform similarly on the training dataset. The MLR model has a slightly lower training error.
- For the test dataset (last 20% of the data), the MLR model again outperforms the Ridge model by a small margin.